In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.stats.diagnostic import acorr_ljungbox
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load the dataset
file_path = '../data/ML471_S2_Datafile_Concept(in).csv'
df = pd.read_csv(file_path)

# Data Preprocessing
df['Datetime'] = pd.to_datetime(df['Datetime'])
df.set_index('Datetime', inplace=True)

# Drop missing values in Power_Consumption_diff
df = df.dropna(subset=['Power_Consumption_diff'])

# Display first few rows
df.head()

In [ ]:
# Train-Test Split (80:20)
train_size = int(len(df) * 0.8)
train, test = df['Power_Consumption_diff'][:train_size], df['Power_Consumption_diff'][train_size:]

print(f"Training data size: {len(train)}")
print(f"Testing data size: {len(test)}")

In [ ]:
# AIC-based Grid Search for AR(p) with p from 0 to 14
aic_values = []
for p in range(15):
    try:
        model = ARIMA(train, order=(p, 0, 0))
        results = model.fit()
        aic_values.append((p, results.aic))
    except:
        continue

# Find the best p
best_p, best_aic = min(aic_values, key=lambda x: x[1])
print(f"Best lag order p: {best_p} with AIC: {best_aic:.4f}")

In [ ]:
# Fit the best AR model (AR(14))
best_model = ARIMA(train, order=(14, 0, 0))
best_results = best_model.fit()

# Display Model Summary
print(best_results.summary())

In [ ]:
# Residual Diagnostics: Ljung-Box Test at Lag 1
residuals = best_results.resid
lb_test = acorr_ljungbox(residuals, lags=[1], return_df=True)
print("Ljung-Box Test at lag 1:")
print(lb_test)

if lb_test['lb_pvalue'].iloc[0] > 0.05:
    print("\nResiduals show no significant autocorrelation (resemble white noise).")
else:
    print("\nResiduals show significant autocorrelation.")

In [ ]:
# Forecasting
forecast = best_results.forecast(steps=len(test))

# Calculate Accuracy Metrics
mae = mean_absolute_error(test, forecast)
rmse = np.sqrt(mean_squared_error(test, forecast))
mape = np.mean(np.abs((test - forecast) / test)) * 100

print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAPE: {mape:.4f}%")

In [ ]:
# Visualization
plt.figure(figsize=(14, 7))
plt.plot(train, label='Training data', color='blue')
plt.plot(test.index, test, label='Actual test data', color='orange', linestyle='--')
plt.plot(test.index, forecast, label='Forecasted values', color='green', linestyle='--')

plt.title('AR(14) Model Forecast vs Actual Data')
plt.xlabel('Date')
plt.ylabel('Power Consumption Diff')
plt.legend()
plt.grid(True)
plt.show()